In [ ]:
!pip install google-play-scraper

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.4/447.4 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/13

In [ ]:
import httpx
def check_app_id(app_url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = httpx.get(app_url, headers=headers)
    return response.status_code == 200

In [ ]:
import re
def extract_app_id(app_url):

  pattern = r'id=([^&]+)'
  match = re.search(pattern, app_url)
  if match:
      app_id = match.group(1)
      return app_id
  else:
      print('Incorrect App ID.')

In [ ]:
def reviews_content(reviews):
  data_list = []
  for i in reviews:
    data_list.append(i['content'])
  return data_list

In [ ]:
import re
def clean_text(text):
  emoji_pattern = re.compile(
    "["
    "\U0001F600-\U0001F64F"  # Emoticons
    "\U0001F300-\U0001F5FF"  # Symbols & pictographs
    "\U0001F680-\U0001F6FF"  # Transport & map symbols
    "\U0001F700-\U0001F77F"  # Alchemical symbols
    "\U0001F780-\U0001F7FF"  # Geometric shapes extended
    "\U0001F800-\U0001F8FF"  # Supplemental arrows-C
    "\U0001F900-\U0001F9FF"  # Supplemental symbols and pictographs
    "\U0001FA00-\U0001FA6F"  # Chess symbols
    "\U0001FA70-\U0001FAFF"  # Symbols and pictographs extended-A
    "\U00002702-\U000027B0"  # Dingbats
    "\U000024C2-\U0001F251"  # Enclosed characters
    "]+",
    flags=re.UNICODE
)

  return emoji_pattern.sub(r'', text)

In [ ]:
import pandas as pd
def clean_df(reviews):

  df = pd.DataFrame(reviews , columns = ['Reviews'])
  df['Reviews'] = df['Reviews'].apply(clean_text)
  empty_rows = df[df['Reviews'] == '']
  df.drop(labels = empty_rows.index , inplace = True)
  return df


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.0 MB/s eta 0:00:00


In [ ]:
from google_play_scraper import Sort, reviews_all , app

def scrape_reviews_content(app_url):     ## Returns the List of Reviews_Content

  if not check_app_id(app_url):
    return 'Invalid App URL'

  app_id = extract_app_id(app_url)

  try:
    result = reviews_all(
      app_id,
      sleep_milliseconds=50, # defaults to 0
      lang='en', # defaults to 'en'
      country='us', # defaults to 'us'
      sort=Sort.NEWEST, # defaults to Sort.NEWEST
    )

    if len(result) == 0:
      return 'No reviews found for this app.'

    else:
      return reviews_content(result)


  except Exception as e:
    print(f'Error Scraping Reviews: {e}')
    return



In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

def huggface_model(model_path):
    sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path , device = 0)
    return sentiment_task

In [ ]:
def df_labels(df , model):
    try:
        label_lst = []
        for i in df['Reviews']:
            label_lst.append(model(i)[0]['label'])
    except Exception as e:
        print(e)

    if label_lst:
        new_df = df.iloc[:len(label_lst)]
        new_df['Label'] = label_lst
    else:
        return "Error during Sentiment"

    return new_df

In [ ]:
model = huggface_model(model_path = 'cardiffnlp/twitter-roberta-base-sentiment-latest')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Gradio UI


In [ ]:
import gradio as gr
from matplotlib import pyplot as plt

def analyze_sentiment(app_url):
    try:
        # Attempt to perform the sentiment analysis
        reviews = scrape_reviews_content(app_url)
        df = clean_df(reviews)
        df_label = df_labels(df, model)

        sentiment_counts = {'positive': 0, 'negative': 0, 'neutral': 0}
        for label in df_label['Label']:
            if label in sentiment_counts:
                sentiment_counts[label] += 1

        # Create a bar chart
        labels = list(sentiment_counts.keys())
        values = list(sentiment_counts.values())
        plt.figure(figsize=(6, 4))
        plt.bar(labels, values, color=['green', 'gray', 'red'])
        plt.title('Sentiment Analysis Results')
        plt.xlabel('Sentiment')
        plt.ylabel('Count')

        # Save the plot to a file
        plt.savefig('sentiment_plot.png')
        plt.close()

        return 'sentiment_plot.png', None  # Return the plot image and no error message

    except Exception as e:
        # Return an error message if something goes wrong
        return None, f"There is something wrong in your URL"

# Gradio interface
with gr.Blocks() as demo:
    gr.Markdown("# Sentiment Analysis Tool")
    url_input = gr.Textbox(label="Enter URL")
    analyze_button = gr.Button("Analyze Sentiment")
    image_output = gr.Image(label="Sentiment Analysis Graph")
    error_output = gr.Textbox(label="Error Message", interactive=False)

    analyze_button.click(analyze_sentiment, inputs=url_input, outputs=[image_output, error_output])

demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7872f66be55096ee36.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Sentiment
Positive    2
Negative    2
Neutral     1
Name: count, dtype: int64